In [1]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from helpers import lemmatize_stemming, get_preprocessed, compute_coherence_values

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline


filenames = !ls data/unzipped
filenames = [ 'data/unzipped/'+filename for filename in filenames ]

import random
# indeces = random.sample(range(len(filenames)), 50)
indeces = [0]

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
from helpers import load_tweets

documents = load_tweets(np.array(filenames)[indeces])

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()


processed_docs = [ get_preprocessed(doc, stemmer, lemmatizer) for doc in documents ]
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=14, id2word=dictionary, passes=10, workers=4, alpha=0.01, eta=.91)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Topic: 0 
Words: 0.009*"protect" + 0.009*"lockdown" + 0.007*"stay" + 0.007*"help" + 0.006*"inform" + 0.006*"coronavirus" + 0.006*"cover" + 0.005*"doctor" + 0.005*"week" + 0.005*"american"
Topic: 1 
Words: 0.010*"state" + 0.009*"case" + 0.008*"true" + 0.008*"coronavirus" + 0.008*"china" + 0.007*"realdonaldtrump" + 0.007*"death" + 0.007*"updat" + 0.006*"stupid" + 0.006*"leader"
Topic: 2 
Words: 0.009*"intellig" + 0.007*"take" + 0.007*"theori" + 0.006*"manmad" + 0.005*"coronavirus" + 0.005*"vaccin" + 0.005*"conspiraci" + 0.005*"debunk" + 0.004*"spread" + 0.004*"gonna"
Topic: 3 
Words: 0.010*"thank" + 0.007*"coronavi

In [ ]:
lda_model.save('trained_lda')

In [ ]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus_tfidf, dictionary)
vis

In [ ]:
len(documents)